Geologic Image Processing in Python
-------------------------------------------------------

As a geoscientist, some of the most useful and frequently-used computational tools fall under the broad category of image processing.  It's more than working with photographs or satellite imagery, though.  All "image processing" means in this context is working with data that's on a regular grid. For example, a digital elevation model is every bit as much an image as a core photograph is. 

This tutorial will introduce you to some core image processing methods by solving a handful of realistic tasks related to geology and geophysics.  The goal is to gain familiarity with key "building blocks" and terminology so that you can understand how to use common Python libraries such as `scipy.ndimage` and `sklearn` in your day-to-day work.  For many of you, these may seem like simple tasks and things that are trivial to accomplish in ArcGIS, ImageJ, or Photoshop.  However, the terminology is a bit different when working with image processing and computer vision libraries.  Many operations are called very different things, or are broken into smaller pieces.  Therefore, it's important to understand how to string the fundamental operations that are usually exposed in programming libraries into the higher-level operations you're used to thinking about.  

Because the Transform2020 tutorial is remote, it's difficult to provide the type of hands-on help we normally would.  Therefore, this set of notebooks is meant to be a "cookbook" demonstrating common tasks and illustrating underlying principles through specific examples.  We won't go over all of the details, but hopefully you can come back to these examples later and adjust them to your needs. 

This tutorial assumes basic Python knowledge and at least some familiarity with `numpy`.  However, if you're new to all of this, that's perfectly okay!  In that case, focus on how the operations are chained together more than the details of the code. If you're looking for tutorials on learning Python for scientific purposes, I'd highly recommend both the "official" python tutorial and Scipy Lectures

One more note is that this is not meant to be a complete introduction to image processing, or even a complete introduction to common geoscience image processing problems. However, hopefully after this tutorial you feel comfortable enough to start experimenting and learning more on your own.

We'll focus on using a combination of `numpy`, `scipy.ndimage`, and `sklearn`. `sklearn` is a leading image processing library that has many very nice features and exposes many advanced methods.  `scipy.ndimage` is a bit more low-level, but has the advantage of both working in 3D (or N-D) and focusing on efficient implementations of common operations.   We'll also use libraries like `rasterio` for reading and writing geospatial data, but we won't dive deeply into the details of working with geospatial data.

Outline...

Why use a grid instead of x,y,z points?
--------------------------------------------------------

This tutorial is going to focus entirely on gridded data.  (That's what we mean by an "image" -- data on a regular grid.)  However, many people are more confortable and familiar with working with data as a series of points; i.e. something you can view and manipulate in a spreadsheet or database.

There's a common misconception that a table of X, Y, Z data is equivalent to a grid.  

Converting from a grid to XYZ points _loses information_.  A grid has inherent structure. We know exactly what pixels are adjacent to each other and don't have to do any extra work to retrieve them.  If we have XYZ points, we have to do a lot of work to determine which points are near each other.

That may sound 


Common types of image data
-------------------------------------------

Diving in: Identifying Seamounts
------------------------------------------------

Let's get started with some concrete examples.  If you'd like to see where we're going, you can jump straight to the complete example and run it:

In [ ]:
%load examples/seamount_detection.py

We're going to try to detect, count, and calculate areas of seamounts based on bathymetry data.  Along the way, we'll cover the following image processing concepts:

  * Array representation
  * Thresholding
  * Filtering
  * Segmentation
  
Let's start by loading our data from a geotiff and taking a look at it:

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import rasterio as rio

from context import data

# Let's load data from a geotiff using rasterio...
with rio.open(data.gebco.seamounts, 'r') as src:
    bathy = src.read(1)
    
print(bathy)

So `bathy` is a 2D array with integer values.  The units are meters relative to sea level.  Note that more or less everything is negative: This is GEBCO bathymetry data from a of the Western Pacific near the Marianas Trench.

Let's take a look at what this data looks like:

In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(bathy, cmap='gray')
fig.colorbar(im, orientation='horizontal')
plt.show()

We have a single array of data that we're displaying as grayscale.  Let's go ahead and add some color to that. We'll discuss color in images in more detail later, but this is a good chance to briefly introduce using colormaps in matplotlib:

In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(bathy, cmap='Blues_r', vmax=0)
im.cmap.set_over('green') # Just display any land as green...
fig.colorbar(im, orientation='horizontal')
plt.show()

Okay, so we're looking at a large number of seamounts rising up above the abyssal plain.  It's really obvious where they are visually, but it would be nice to be able to quickly identify them programatically.  For example, we might want to look at their distribution by area or volume, or to just get a count without manually counting all of them.

Thresholding
-------------------

The simplest approach we could take would be to threshold the bathymetry data.  The abyssal plain is usually around 4km depth due to the relatively constant thickness and density of oceanic crust.  Therefore, we could try thresholding out anything above 3500 meters.  Because this is a numpy array, the operation is quite simple:

In [ ]:
simple_threshold = bathy > -3500
print(simple_threshold)

Note that we have an array of True/False values. This is a boolean array, and some of the operations we'll work with today only operate on these sort of boolean True/False arrays. 

Often, in image processing, you'll convert the True/False representation into a 1/0 representation.  Behind the scenes, the `True` values above can be efficiently converted into `1` and the `False` values into `0`. For example:

In [ ]:
# "view" only changes the way we're interepting the underlying data.
# If you're not familiar with "view" vs "astype", use "astype". All I'm
# showing here is that it's seamless to go from True/False --> 1/0
print(simple_threshold.view(np.uint8)) 

Okay, enough beating around the bush. Let's take a look at what we've accomplished:

In [ ]:
fig, axes = plt.subplots(nrows=2, sharex=True, sharey=True)
axes[0].imshow(bathy, cmap='Blues_r', vmax=0)
axes[1].imshow(simple_threshold.view(np.uint8))

for ax in axes.flat:
    ax.set(xticks=[], yticks=[])
fig.tight_layout()

plt.show()

The yellow regions are `True` in the boolean array. Note that we capture many seamounts out in the abyssal plain, but classify the entire volcanic arc and forearc in the west as a single large seamount.  We also miss a lot of smaller seamounts that 

Let's take a second to make a fancier display so we can explore what we capture and what we don't.  I'm going to use a quick utility included with this tutorial to allow toggling of different overlays on the plot.  We'll re-use this throughout the tutorial.

In [ ]:
from context import utils

fig, ax = plt.subplots()
ax.imshow(bathy, cmap='Blues_r', vmax=0).cmap.set_over('green') 

# We'll mask any False values so that they're transparent
im = ax.imshow(np.ma.masked_where(~simple_threshold, simple_threshold),
               vmin=0, vmax=1, label='>3500 mbsl')

ax.set(xticks=[], yticks=[])
fig.tight_layout()

utils.Toggler(im).show()

As you can see, we're doing an okay job of detecting large seamounts, but an awful job of detecting smaller ones and arc volcanoes.  This is because we're using a fixed elevation threshold to determine whether or not something is a seamount.

Visually, we'd determine whether or not a pixel is part of a seamount based on the area around it. We're looking for features that rise up from the surrounding topography.  However, that "base level" of topography varies throughout our study area.  Therefore we need a way of finding the "background" elevation.  Remember that -- we'll come back to it soon.

Filters
---------

Filters (and convolution, which is verly closely related) are an ubiquitous concept in image processing.  Filtering an image is a type of "moving window" operation.  For each pixel in the image, we take a region around it and apply some operation based on that region to define a new pixel value.  Most commonly used filters involve multiplying each pixel in the region by a weight and then summing (i.e. a convoluion).  This allows blurring, sharpening, edge detection, and many other useful operations.  Other filers aren't defined by weights, but by more flexible operations.  A simple example of this is a median filter, where the value of the pixel is the median of the pixels in some window surrounding it.  To calculate a median, we need to sort all the pixels we're using and find the one in the middle -- it can't be defined by a weighted average.

Let's use one of the simplest possible filters: a uniform filter.  A uniform filter is an average of all pixel values in a square region.  It's simple, but it's fast.  In practice, it blurs the result:

In [ ]:
import scipy.ndimage

# Need to use this to show impact of filter width...

window = 100
blurred = scipy.ndimage.uniform_filter(bathy, window)

fig, ax = plt.subplots()
ax.imshow(bathy, cmap='Blues_r', vmax=0)
im = ax.imshow(blurred, cmap='Blues_r', vmin=bathy.min(), vmax=0, label='Blurred')

ax.set(xticks=[], yticks=[])
fig.tight_layout()

utils.Toggler(im).show()

Using filters in seamount detection
---------------------------------------------------

Going back to our seamount detection problem, let's use the blurred / uniform-filtered bathymetry to define the "background" elevation.  Seamounts or other peak-like features will be significantly higher than the background elevation and trenches or other trough-like features will be significantly below it.  Therefore, we can identify seamounts by comparing the uniform-filtered bathymetry data to the original bathymetry data.  Anything that's more than some amount higher than the background elevation, we'll consider a seamount: